In [7]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append('../src')

from data import tone_analyzer

%aimport data.tone_analyzer

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from dotenv import load_dotenv
load_dotenv()
import os
import requests
import time
import textrazor

In [9]:
from pymongo import MongoClient
import numpy as np

MONGO_PASSWORD = os.environ['MONGO_USER_PASSWORD']
CROWD_TANGLE = os.environ['CROWD_TANGLE_API_TOKEN']

config = {
  'host': '18.216.52.13:27017',
  'username': 'mongo_user',
  'password': MONGO_PASSWORD,
  'authSource': 'reddit_climate_news'
}

db = MongoClient(**config).reddit_climate_news

# Crowd Tangle

## FB Interactions

In [19]:
def get_crowd_tangle(link, ct_token):
    try:
        ct_inter_req = f'https://api.crowdtangle.com/ce/interactions?link={link}&token={ct_token}'
        inter_resp = requests.get(ct_inter_req)
        return inter_resp.json()
    except KeyboardInterrupt:
        raise
    except Exception as e:
        print(str(e))
        return { 'error': str(e)}

In [20]:
count = 0
for record in db.climate_submissions.find({'crowd_tangle': {'$exists': False}}, {'_id': 1, 'url': 1}):
    url = record['url']
    ct_res = get_crowd_tangle(url, CROWD_TANGLE)
    print(count)
    db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'crowd_tangle': ct_res}})
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [26]:
codes = np.array([x['crowd_tangle']['status'] for x in db.climate_submissions.find() if 'crowd_tangle' in x])

In [33]:
np.unique(codes, return_counts=True)

(array([200, 400, 401]), array([1571,   96,   16]))

## Reddit Activity

In [39]:
def get_reddit_crowd_tangle(link, ct_token):
    try:
        ct_req = f'https://api.crowdtangle.com/ce/links?link={link}&platforms=reddit&token={ct_token}&version=3.0.27'
        resp = requests.get(ct_req)
        return resp.json()
    except KeyboardInterrupt:
        raise
    except Exception as e:
        print(str(e))

In [40]:
count = 0
for record in db.climate_submissions.find():
    url = record['url']
    ct_res = get_reddit_crowd_tangle(url, CROWD_TANGLE)
    if count % 100 == 0:
        print(count)
    if ct_res:
        db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'reddit_crowd_tangle': ct_res}})
        count += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
Expecting value: line 1 column 1 (char 0)
1700
1800
1900
2000
2100
2200


# Text Razor

In [32]:
textrazor.api_key = os.environ['TEXT_RAZOR_TWO']

client = textrazor.TextRazor(extractors=["entities", "topics"])
client.set_cleanup_return_cleaned(True)

count = 0
error_count = 0

for record in db.climate_submissions.find({'text_razor': {'$exists': False}}).limit(200):
    try:
        url = record['url']
        res = client.analyze_url(url)
        if count % 100 == 0:
            print(error_count)
            print(count)
        db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'text_razor': res.json}})
        count += 1
    except KeyboardInterrupt as e:
        raise e
    except textrazor.TextRazorAnalysisException as e:
#         db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'text_razor': str(e)}})
        count += 1
        print(e)
        error_count += 1

0
0
0
100
TextRazor returned HTTP Code 401: b'{"ok":false, "time":0, "error":"You have reached your daily TextRazor request limit. Please contact support@textrazor.com for more information, or visit https://www.textrazor.com to upgrade your account."}\n\n'
TextRazor returned HTTP Code 401: b'{"ok":false, "time":0, "error":"You have reached your daily TextRazor request limit. Please contact support@textrazor.com for more information, or visit https://www.textrazor.com to upgrade your account."}\n\n'
TextRazor returned HTTP Code 401: b'{"ok":false, "time":0, "error":"You have reached your daily TextRazor request limit. Please contact support@textrazor.com for more information, or visit https://www.textrazor.com to upgrade your account."}\n\n'
TextRazor returned HTTP Code 401: b'{"ok":false, "time":0, "error":"You have reached your daily TextRazor request limit. Please contact support@textrazor.com for more information, or visit https://www.textrazor.com to upgrade your account."}\n\n'
Te

In [92]:
db.climate_submissions.find({'text_razor': {'$exists': False}}).count()

/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


2068

In [93]:
db.climate_submissions.find({'text_razor': {'$exists': True}}).count()

/Users/willyraedy/opt/anaconda3/envs/metis/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.


211

# Tone

In [6]:
count = 0
for record in db.climate_submissions.find({'tone_analyzer': {'$exists': False}}, {'title': 1}):
    try:
        title = record['title']
        res = tone_analyzer.get(title)
        if count % 100 == 0: 
            print(count)
        db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'tone_analyzer': res}})
    #     time.sleep(0.2)
        count += 1
    except KeyboardInterrupt:
        raise
    except Exception as e:
        print(e)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400


# Sentiment

In [35]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [37]:
analyzer = SentimentIntensityAnalyzer()

count = 0
for record in db.climate_submissions.find():
    title = record['title']
    res = analyzer.polarity_scores(title)['compound']
    if count % 100 == 0: 
        print(count)
    db.climate_submissions.update_one({'_id': record['_id']}, {'$set': {'sentiment_score': res}})
    count += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


In [50]:
db.climate_submissions.find_one()

{'_id': ObjectId('5e65d004bd71f87e9891c891'),
 'all_awardings': [],
 'allow_live_comments': False,
 'author': 'PBR--Streetgang',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_nssruw2',
 'author_patreon_flair': False,
 'author_premium': True,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1583573838,
 'domain': 'cnn.com',
 'full_link': 'https://www.reddit.com/r/worldnews/comments/fet0fd/these_south_african_farmers_say_their_cows_can/',
 'gildings': {},
 'id': 'fet0fd',
 'is_crosspostable': True,
 'is_meta': False,
 'is_original_content': False,
 'is_reddit_media_domain': False,
 'is_robot_indexable': True,
 'is_self': False,
 'is_video': False,
 'link_flair_background_color': '',
 'link_flair_richtext': [],
 'link_flair_text_color': 'dark',
 'link_flair_type': 'text',
 'locked': False,
 'media_only': False,
 'no_follow': True,
 'num_comments': 1,
 'num_cro